# Google "News" tab web scraping

## I decided to collect first 10 pages of news about the olimpic games 2022 in google's news tab (Russian language)

### This can be really useful if we want to analyze news topics (for example, how different news affilates tell people about ups and downs of their country or what they write about any doping scandals)

### Of course, this code is universal in some way: to collect the information about others topics (or change the amount of loaded data) you just need to change some symbols in this code

In [3]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm
import pandas as pd

## Chrome driver installing

In [4]:
s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://www.google.com/search?q=олимпиада&sxsrf=APq-WBvDA7edewa5jBhoh_R3-274Fe9vAQ:1644488930864&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjBk77U9vT1AhXksIsKHWOHAXUQ_AUoAXoECAIQAw"
browser.get(url)

## Saving "cards" with necessary data

In [5]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [6]:
cards = soup.findAll("g-card", class_ = "ftSUBd")

In [8]:
len(cards)

9

## Analyzing the first card

In [9]:
card = cards[0]

### getting resource's name

In [15]:
comp = card.find("div", class_ = "CEMjEf NUnG9d").find("span").text.strip()

In [24]:
comp

'Чемпионат'

### getting the topic

In [22]:
topic = card.find("div", class_ = "iRPxbe").findAll("div")[1].text.strip()

In [23]:
topic

'Валиеву и золото вроде бы отстояли. Но нужны официальные подтверждения. \nОнлайн'

### getting the link to this article

In [25]:
link = card.find("a", class_ = "WlydOe").get("href")

In [26]:
link

'https://www.championat.com/olympicwinter/article-4603603-doping-proba-rossii-v-figurnom-katanii-na-olimpijskih-igrah-2022-onlajn-vse-podrobnosti-poslednie-novosti.html'

### clicking on "next page" button

In [27]:
browser.find_element(By.CSS_SELECTOR, "#pnnext > span:nth-child(2)").click()

# Final code

In [28]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm
import pandas as pd

data = []

s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://www.google.com/search?q=олимпиада&sxsrf=APq-WBvDA7edewa5jBhoh_R3-274Fe9vAQ:1644488930864&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjBk77U9vT1AhXksIsKHWOHAXUQ_AUoAXoECAIQAw"
browser.get(url)

for i in tqdm(range(1, 11)):
    sleep(5)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    cards = soup.findAll("g-card", class_ = "ftSUBd")
    
    for card in cards:
        try:
            company = card.find("div", class_ = "CEMjEf NUnG9d").find("span").text.strip()
        except:
            company = "-"
        try:
            topic = card.find("div", class_ = "iRPxbe").findAll("div")[1].text.strip()
        except:
            topic = "-"
        try:
            link = card.find("a", class_ = "WlydOe").get("href")
        except:
            link = "-"
        data.append([company, topic, link])
        
    try:
        browser.find_element(By.CSS_SELECTOR, "#pnnext > span:nth-child(2)").click()
    except:
        break

headers = ["company", "topic", "link"]
df = pd.DataFrame(data, columns=headers)

df.to_json("./news.json", orient = 'index')

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]
